In [2]:
import pickle
import numpy as np
import tensorflow as tf

def pkl_to_tensor(path):
    img_lis=[]
    lable =[]
    with open(path, 'rb') as f:
        data = pickle.load(f)
        feature = np.array(data['image_array'])
        for data in feature:
            #print(data[2])
            #tensor = transform(data[2])
            
            X = np.expand_dims(data[2], axis=2)
            #print(X.shape)
            X = X.astype('float32')
            # scale from [0,255] to [0,1]
            X = X / 255.0
            #print(asarray(X).shape)
            img_lis.append([X])
            if "positive" in path:

                lable.append(1)
            else:
                lable.append(0)
            # fe_tensor = tf.convert_to_tensor(feature)
            # L.append(fe_tensor)

    Butch_ten=tf.concat(img_lis, axis=0)
    lable=np.array(lable,dtype="uint8")
    rgb_batch = np.repeat(Butch_ten[:,:,:,0][..., np.newaxis], 3, -1)

    return rgb_batch,lable

In [3]:
Butch_ten_pos,lable_pos=pkl_to_tensor(r"C:\Users\Thenu\Downloads\cc_positive_11.pkl")
Butch_ten_neg,lable_neg=pkl_to_tensor(r"C:\Users\Thenu\Downloads\cc_positive_11.pkl")

C:\Users\Thenu\AppData\Local\Temp\ipykernel_6500\599577646.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  feature = np.array(data['image_array'])


In [4]:
import keras
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.applications import ResNet50

def model():
    model  = Sequential()
    model.add(ResNet50(
        include_top=False,
        pooling='avg',
        weights='imagenet',
        ))

    model.add(Dense(1, activation='softmax'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [5]:
model = model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 23,589,761
Trainable params: 23,536,641
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
n_epochs=4
n_batch=2
bat_per_epo = int(Butch_ten_pos.shape[0] / n_batch)

for i in range(n_epochs):
		# enumerate batches over the training set
	for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			#X, y = np.vstack((Butch_ten_pos[j*n_batch:(j+1)*n_batch,:,:,:], lable_pos[j*n_batch:(j+1)*n_batch])), np.vstack((Butch_ten_pos[j*n_batch:(j+1)*n_batch,:,:,:], lable_pos[j*n_batch:(j+1)*n_batch]))
			# update discriminator model weights
		_, real_acc_pos = model.train_on_batch(Butch_ten_pos[j*n_batch:(j+1)*n_batch,:,:,:], lable_pos[j*n_batch:(j+1)*n_batch])
		_, real_acc_neg = model.train_on_batch(Butch_ten_neg[j*n_batch:(j+1)*n_batch,:,:,:], lable_neg[j*n_batch:(j+1)*n_batch])

		print((real_acc_neg+real_acc_pos)/2)

	filename = 'generator_model_%03d.h5' % (i + 1)
	model.save(filename)